# MoMa's collection data cleaning

## Problem solving
I'm working for the MoMa and they'd like to know in which department they need to enrich based on the current collection. 

**What is the Top-3 less valuable classification?**

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('data/museum_modern_art.csv',sep=',')
df.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN


## Renaming columns

In [10]:
df=df.rename(columns={'Unnamed: 0':'Id'})

In [11]:
df.columns

Index(['Id', 'Title', 'Artist', 'ConstituentID', 'ArtistBio', 'Nationality',
       'BeginDate', 'EndDate', 'Gender', 'Date', 'Medium', 'Dimensions',
       'CreditLine', 'AccessionNumber', 'Classification', 'Department',
       'DateAcquired', 'Cataloged', 'ObjectID', 'URL', 'ThumbnailURL',
       'Circumference (cm)', 'Depth (cm)', 'Diameter (cm)', 'Height (cm)',
       'Length (cm)', 'Weight (kg)', 'Width (cm)', 'Seat Height (cm)',
       'Duration (sec.)'],
      dtype='object')

## Drop empty and useless tables

In [15]:
null_col=df.isna().sum()
null_col_percent=round(null_col[null_col>0]/df.shape[0]*100,2)
null_col_percent

Title                   0.03
Artist                 12.20
ConstituentID          12.20
ArtistBio              14.81
Nationality            12.20
BeginDate              12.20
EndDate                12.20
Gender                 12.20
Date                   12.84
Medium                 18.64
Dimensions             18.40
CreditLine             13.14
AccessionNumber        11.26
Classification         11.26
Department             11.26
DateAcquired           15.62
Cataloged              11.26
ObjectID               11.26
URL                    49.17
ThumbnailURL           56.12
Circumference (cm)     99.99
Depth (cm)             91.45
Diameter (cm)          99.07
Height (cm)            23.75
Length (cm)            99.52
Weight (kg)            99.81
Width (cm)             24.34
Seat Height (cm)      100.00
Duration (sec.)        97.93
dtype: float64

In [21]:
drop_cols=null_col_percent[null_col_percent>50].index
new_df=df.drop(drop_cols,axis=1)

new_df.head()

,Id,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,Height (cm),Width (cm)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,...,Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,48.6000,168.9000
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,...,Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,40.6401,29.8451
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,34.3000,31.8000
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,...,Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,50.8000,50.8000
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,38.4000,19.1000


# Put off parenthesis on text in relevant columns

In [48]:
parenthesis_col=new_df[['ArtistBio','Nationality','BeginDate','EndDate','Gender']].columns
parenthesis_col

Index(['ArtistBio', 'Nationality', 'BeginDate', 'EndDate', 'Gender'], dtype='object')

In [56]:
for col in parenthesis_col:
    new_df[col]=new_df[col].str.replace('\(','').str.replace('\)','')
    
new_df.head()

,Id,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,Height (cm),Width (cm)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"Austrian, 1841–1918",Austrian,1841,1918,Male,1896,...,Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,48.6000,168.9000
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"French, born 1944",French,1944,0,Male,1987,...,Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,40.6401,29.8451
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"Austrian, 1876–1957",Austrian,1876,1957,Male,1903,...,Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,34.3000,31.8000
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"French and Swiss, born Switzerland 1944",,1944,0,Male,1980,...,Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,50.8000,50.8000
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"Austrian, 1876–1957",Austrian,1876,1957,Male,1903,...,Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,38.4000,19.1000


## Remove Duplicates

In [32]:
new_df.iloc[:,:-1]

,Id,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,...,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,Height (cm)
0,0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,...,"19 1/8 x 66 1/2"" (48.6 x 168.9 cm)",Fractional and promised gift of Jo Carole and ...,885.1996,Architecture,Architecture & Design,1996-04-09,Y,2,http://www.moma.org/collection/works/2,48.6000
1,1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,...,"16 x 11 3/4"" (40.6 x 29.8 cm)",Gift of the architect in honor of Lily Auchinc...,1.1995,Architecture,Architecture & Design,1995-01-17,Y,3,http://www.moma.org/collection/works/3,40.6401
2,2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,"13 1/2 x 12 1/2"" (34.3 x 31.8 cm)",Gift of Jo Carole and Ronald S. Lauder,1.1997,Architecture,Architecture & Design,1997-01-15,Y,4,http://www.moma.org/collection/works/4,34.3000
3,3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,...,"20 x 20"" (50.8 x 50.8 cm)",Purchase and partial gift of the architect in ...,2.1995,Architecture,Architecture & Design,1995-01-17,Y,5,http://www.moma.org/collection/works/5,50.8000
4,4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,...,"15 1/8 x 7 1/2"" (38.4 x 19.1 cm)",Gift of Jo Carole and Ronald S. Lauder,2.1997,Architecture,Architecture & Design,1997-01-15,Y,6,http://www.moma.org/collection/works/6,38.4000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152482,135313,Lost Portraits (Double),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,F2016.86.2,Film,Film,NaN,N,290213,NaN,NaN
152483,135314,"Lost Portraits (Group) and Ula, Charlie and Fr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,F2016.86.3,Film,Film,NaN,N,290214,NaN,NaN
152484,135315,Hilde,Maria Lassnig,3402,"(Austrian, 1919–2014)",(Austrian),(1919),(2014),(Female),1972-1976,...,NaN,NaN,820159.1,Film,Film,NaN,N,290234,NaN,NaN
152485,135316,Bärbl,Maria Lassnig,3402,"(Austrian, 1919–2014)",(Austrian),(1919),(2014),(Female),1973,...,NaN,NaN,820159.2,Film,Film,NaN,N,290235,NaN,NaN
